In [191]:
import numpy as np
np.random.seed(4)
#gaussian distribution sampler: normal(mean=0.0, variance_sqrt=1.0, size=None) s = np.random.normal(0,1)
#teacher model
#randomly generate z=(x,y) with x evenly sampled from (0,10) and y = 1.5 x + 5, a random noise is added. A total of N samples are drawn.
#dimension of x: d
d = 10
#number of training samples
N = 2000
#dimension of hypothesis space
p = 10

#point-wise activate function f:tanh
#variance of random noise added to y
sigma = 0.1
#samples X(N*d)
X = np.random.normal(0,1,(N,d))
#random feature matrix F(d*p)
F = np.random.normal(0,1,(d,p))
#teacher parameter w(p) with lambda = ? until each dim of Y~1e0
lambda_ = 0.000001
w_0 = np.random.normal(0,sigma**2/lambda_/N,p)
#X after the random feature matrix
X_rf = np.tanh(X.dot(F)/np.sqrt(d))
Y_pure = X_rf.dot(w_0)
Y = Y_pure + np.random.normal(0,sigma**2,N)
print(Y_pure)
print(Y)

[-7.46531452  3.5784489  -6.39292894 ...  6.56196418 -1.86685126
  3.70851177]
[-7.45996833  3.57295138 -6.4065695  ...  6.58191236 -1.86125286
  3.69766249]


In [192]:
#empirical risk
#L_S(w)
def L_S(w):
    diff = Y-X_rf.dot(w)
    Nloss = diff.dot(diff)
    return Nloss/N

def grad_L_S(w):
    return -2/N*X_rf.T.dot(Y-X_rf.dot(w))

grad_L_S(w_0+0.1)#when N is small, diffusion.

array([-0.02118863,  0.15274631,  0.03218446,  0.15062137,  0.1599449 ,
        0.17776951,  0.15951735,  0.05462499,  0.04963837, -0.02631697])

In [193]:
#minus log distribution: f
#prar beta(also change h if change this)
beta = 1000
#para sigma_q, can be set according to N.
sigma_q = 1
def f(w): 
    return beta*L_S(w)+(1/2/sigma_q**2)*(w.dot(w))

def grad_f(w):
    grad = beta*grad_L_S(w)+1/sigma_q**2*w
    return grad

In [194]:
#MCMC
#stepsize h(h*beta=0.01)
h = 0.00001
w = np.zeros(p)
for i in range(50000):
    grad_f_w = grad_f(w)
    proposal_state = w-h*grad_f_w+np.random.normal(0,2*h,p)
    reject_thresh = min(1,np.exp(f(w)-f(proposal_state)+(1/4/h)*(np.linalg.norm(proposal_state-w+h*grad_f_w)**2-np.linalg.norm(w-proposal_state+h*grad_f(proposal_state))**2)))
    U = np.random.rand(1)
    if U <= reject_thresh:
        w = proposal_state
print(w)
print(w_0)

[-1.45008745 -6.23232396  0.92529813  5.33051731  3.26004393 -1.28420206
  4.25861759 -1.02099803 -1.75181107 -1.6217237 ]
[-1.47924901 -6.27862105  0.91154059  5.36454552  3.26175221 -1.27554012
  4.24965455 -1.01369899 -1.74258052 -1.64301827]
